In [25]:
import numpy as np
import graph_tool.all as gt

In [42]:
# HELPER FUNCTIONS

def softmax_scaling(data, axis=1):
    """
    Softmax transformation: exp(x) / sum(exp(x))
    Converts scores to probabilities that sum to 1 along specified axis
    """
    exp_scores = np.exp(data)
    return exp_scores / np.sum(exp_scores, axis=axis, keepdims=True)

_Thus, respondents were asked to state, on a scale from 1 to 7, how uncomfortable/comfortable they would feel to be friends with someone who supports a specific political party._ 

Data from Grönlund & Strandberg 2023

In [27]:
# Convert table in Grönlund & Strandberg 2023 to array
scores = np.array([
    [6.3, 2.5, 3.7, 3.9, 5.2, 5.6, 5.0, 3.1, 2.8],
    [3.8, 6.0, 5.0, 4.3, 2.7, 2.7, 3.6, 4.4, 5.0],
    [4.8, 4.3, 6.0, 5.1, 4.4, 3.4, 5.2, 4.7, 4.8],
    [4.4, 4.6, 5.2, 6.2, 3.3, 3.5, 4.7, 4.9, 4.0],
    [6.2, 2.1, 4.8, 4.2, 6.6, 6.3, 6.0, 3.4, 3.6],
    [5.8, 2.4, 3.4, 3.7, 5.4, 6.2, 4.8, 2.9, 3.4],
    [4.7, 2.8, 4.6, 4.0, 4.1, 3.5, 6.2, 3.7, 3.3],
    [4.3, 5.1, 5.0, 5.4, 2.8, 3.1, 4.6, 6.1, 4.8],
    [5.4, 5.6, 5.1, 4.2, 3.0, 3.1, 4.8, 5.0, 5.7]
])

# Label encoding
party_dict = {
    'SDP': 0,
    'PS': 1,
    'KOK': 2,
    'KESK': 3,
    'VIHR': 4,
    'VAS': 5,
    'RKP': 6,
    'KD': 7,
    'LIIKE': 8
}

# Party support from parliamentary elections 2023
party_support_dict = {
    'KOK': 0.208,
    'PS': 0.201,
    'SDP': 0.199,
    'KESK': 0.113,
    'VIHR': 0.07,
    'VAS': 0.071,
    'RKP': 0.043,
    'KD': 0.042,
    'LIIKE': 0.024
    }

# Normalize
total = sum(party_support_dict.values())
party_support_norm_dict = {k: v/total for k, v in party_support_dict.items()}

# Convert to lists for sampling
items = list(party_support_norm_dict .keys())
probabilities = list(party_support_norm_dict .values())

# Row/column order: SDP, PS, KOK, KESK, VIHR, VAS, RKP, KD, LIIK

# Scale scores to softmax, other scalings possible!
softmax_probs = softmax_scaling(scores)

In [32]:
# Set number of partisans
total_partisans = 200
mean_encounters = 20

# Create a room with n partisans
g = gt.Graph()

# Add partisans at once 
vertices = g.add_vertex(total_partisans)

# Create vp for storing party affiliation that are sampled
party_prop = g.new_vertex_property("string")

# Sample affiliations
sampled_affs = [np.random.choice(items, p=probabilities) for _ in range(total_partisans)]
    
# Set vp to the graph
for v in g.vertices():
    party_prop[v] = sampled_affs[int(v)]

# Make it an internal property of the graph if you need to keep it
g.vertex_properties["affiliation"] = party_prop

In [33]:
friendships = []
for v in g.vertices():
    
    source_partisan = int(v)
    
    # Sample the number of encounters
    sampled_encounters = np.random.poisson(mean_encounters)
    
    print(f"Partisan afilliated with {g.vp["affiliation"][v]} is looking for {sampled_encounters} friends...")
    
    # Sample the candidates
    candidate_friends = np.random.choice(np.arange(total_partisans), size=sampled_encounters, replace=True)
    
    for target_partisan in candidate_friends:
   
        if np.random.random() < softmax_probs[party_dict[g.vp["affiliation"][source_partisan]], party_dict[g.vp["affiliation"][target_partisan]]]:
            friendships.append([source_partisan, target_partisan])
            g.add_edge(source_partisan, target_partisan)

Partisan afilliated with RKP is looking for 22 friends...
Partisan afilliated with SDP is looking for 22 friends...
Partisan afilliated with RKP is looking for 17 friends...
Partisan afilliated with VAS is looking for 20 friends...
Partisan afilliated with PS is looking for 18 friends...
Partisan afilliated with SDP is looking for 21 friends...
Partisan afilliated with PS is looking for 29 friends...
Partisan afilliated with VAS is looking for 14 friends...
Partisan afilliated with PS is looking for 25 friends...
Partisan afilliated with KESK is looking for 19 friends...
Partisan afilliated with RKP is looking for 23 friends...
Partisan afilliated with KD is looking for 27 friends...
Partisan afilliated with KOK is looking for 17 friends...
Partisan afilliated with PS is looking for 17 friends...
Partisan afilliated with KOK is looking for 21 friends...
Partisan afilliated with VAS is looking for 17 friends...
Partisan afilliated with KOK is looking for 17 friends...
Partisan afilliate

In [41]:
# Create a color mapping that roughly corresponds to traditional Finnish party colors
# Colors are in RGBA format (Red, Green, Blue, Alpha)
# Colors taken from YLE News article on polls
color_mapping = {
    'KOK': [0.0, 0.28, 0.67, 0.8],    # Rich blue
    'SDP': [1.0, 0.0, 0.2, 0.8],      # Bright red
    'KESK': [0.13, 0.55, 0.13, 0.8],  # Forest green
    'PS': [0.0, 0.75, 1.0, 0.8],      # Light blue/cyan
    'VAS': [0.55, 0.0, 0.0, 0.8],     # Deep burgundy
    'VIHR': [0.56, 0.93, 0.56, 0.8],  # Bright lime green
    'RKP': [1.0, 0.84, 0.0, 0.8],     # Golden yellow
    'KD': [0.58, 0.44, 0.86, 0.8],    # Purple
    'LIIKE': [1.0, 0.41, 0.71, 0.8]   # Magenta/pink
}

# Create a vertex property map for colors
vertex_colors = g.new_vertex_property("vector<double>")

# Assign colors based on party affiliations
for v in g.vertices():
    # Get the party affiliation from the vertex property
    party = g.vertex_properties["affiliation"][v]
    # Assign the corresponding color
    vertex_colors[v] = color_mapping[party]

# Create a sophisticated visualization using force-directed layout
# The sfdp_layout algorithm will help separate different groups naturally
pos = gt.sfdp_layout(g, C=0.8)
#pos = gt.fruchterman_reingold_layout(g, n_iter=1000)   

# Create the visualization with enhanced settings
gt.graph_draw(
    g,
    pos=pos,
    vertex_fill_color=vertex_colors,
    vertex_size=15,                    # Slightly smaller nodes for less overlap
    vertex_color=[0, 0, 0, 1],        # Black border for better visibility
    edge_pen_width=0.8,               # Thinner edges to reduce visual clutter
    output_size=(1000, 1000),         # Larger size for better detail
    bg_color=[1, 1, 1, 1],           # White background
    output="finnish_politics_graph.pdf"
)

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7f2a97de1640, at 0x7f2a8fba0860>

In [37]:
# Create a legend using matplotlib to help interpret the visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Create a figure with appropriate size for 9 parties
fig, ax = plt.subplots(figsize=(8, 4))

# Create legend patches for each party
legend_elements = []
for party, color in color_mapping.items():
    legend_elements.append(
        patches.Patch(
            facecolor=color[:3],
            alpha=color[3],
            label=f'{party} ({party_dict[party]})'  # Include the numeric ID
        )
    )

# Add legend with two columns for better space usage
ax.legend(handles=legend_elements, 
         loc='center', 
         ncol=2,
         title="Finnish Political Parties")
ax.axis('off')
plt.savefig('finnish_politics_legend.pdf', 
            bbox_inches='tight', 
            dpi=300)
plt.close()